In [2]:
import gymnasium as gym

In [6]:
env = gym.make("InvertedPendulum-v5")
obs_space = env.observation_space.shape[0]
act_space = env.action_space.shape[0]
print(f"This environment has {obs_space} continuous state observations "
      f"and {act_space} continuous action")

This environment has 4 continuous state observationsand 1 continuous action


Box(-inf, inf, (4,), float64)